In [3]:
! pip install datasets transformers accelerate 'transformers[torch]' -U huggingface_hub torch



ERROR: Invalid requirement: "'transformers[torch]'"


In [7]:
!pip install torch

  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/d3/1d/a257913c89572de61316461db91867f87519146e58132cdeace3d9ffbe1f/torch-2.3.1-cp311-cp311-win_amd64.whl.metadata
  Using cached torch-2.3.1-cp311-cp311-win_amd64.whl.metadata (26 kB)
  Obtaining dependency information for mkl<=2021.4.0,>=2021.1.1 from https://files.pythonhosted.org/packages/fe/1c/5f6dbf18e8b73e0a5472466f0ea8d48ce9efae39bd2ff38cebf8dce61259/mkl-2021.4.0-py2.py3-none-win_amd64.whl.metadata
  Using cached mkl-2021.4.0-py2.py3-none-win_amd64.whl.metadata (1.4 kB)
  Obtaining dependency information for intel-openmp==2021.* from https://files.pythonhosted.org/packages/6f/21/b590c0cc3888b24f2ac9898c41d852d7454a1695fbad34bee85dba6dc408/intel_openmp-2021.4.0-py2.py3-none-win_amd64.whl.metadata
  Using cached intel_openmp-2021.4.0-py2.py3-none-win_amd64.whl.metadata (1.2 kB)
  Obtaining dependency information for tbb==2021.* from https://files.pythonhosted.org/packages/7b/2d/1e1c70fae8ace2

ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\Sai Teja\anaconda3\Lib\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher
    yield
  File "C:\Users\Sai Teja\anaconda3\Lib\site-packages\pip\_vendor\urllib3\response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
           ^^^^^^^^^^^^^^^^^^
  File "C:\Users\Sai Teja\anaconda3\Lib\site-packages\pip\_vendor\urllib3\response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
           ^^^^^^^^^^^^^^^^^^
  File "C:\Users\Sai Teja\anaconda3\Lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 90, in read
    data = self.__fp.read(amt)
           ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Sai Teja\anaconda3\Lib\http\client.py", line 466, in read
    s = self.fp.read(amt)
        ^^^^^^^^^^^^^^^^^
  File "C:\Users\Sai Teja\anaconda3\Lib\socket.py", line 706, in readinto
    return self._sock.recv_into(b)
        

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [1]:
import pandas as pd

In [2]:
import torch

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)} is available.")
else:
    print("No GPU available. Training will run on CPU.")


No GPU available. Training will run on CPU.


In [3]:
dataset=pd.read_csv('Dataset/Xsum train data 1.csv')

In [4]:
dataset=dataset.dropna(axis=0)

In [5]:
from transformers import AutoModelForSeq2SeqLM,AutoTokenizer


C:\Users\Sai Teja\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
model_path='fine-tuned-abstractive'
model=AutoModelForSeq2SeqLM.from_pretrained(model_path)
tokenizer=AutoTokenizer.from_pretrained(model_path)

In [7]:
from datasets import Dataset
dataset=Dataset.from_pandas(dataset)

In [8]:
train_dataset=dataset

In [9]:
def preprocess(batch):

  inputs=['summarize: '+ doc for doc in batch['document']]
  targets=batch['summary']
  model_inputs=tokenizer(inputs,max_length=1024,truncation=True,padding='max_length',return_tensors='pt')
  with tokenizer.as_target_tokenizer():
    labels=tokenizer(targets,max_length=128,truncation=True,padding='max_length',return_tensors='pt')
  model_inputs['labels']=labels['input_ids']
  return model_inputs



In [10]:
train_dataset_encoded=train_dataset.map(preprocess,batched=True,remove_columns=['id','summary','document'])

Map:   0%|          | 0/9998 [00:00<?, ? examples/s]C:\Users\Sai Teja\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\transformers\tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 9998/9998 [00:16<00:00, 618.39 examples/s]


In [11]:
from transformers import TrainingArguments,Trainer

In [12]:
args=TrainingArguments(output_dir='output/',evaluation_strategy='epoch',num_train_epochs=1,per_device_train_batch_size=8,per_device_eval_batch_size=8,weight_decay=0.01,logging_steps=10,push_to_hub=False)

C:\Users\Sai Teja\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [13]:
trainer=Trainer(model=model,args=args,train_dataset=train_dataset_encoded,tokenizer=tokenizer,eval_dataset=train_dataset_encoded)

In [15]:
trainer.train()

  100%|██████████| 1250/1250 [06:11<10:41:42, 31.05s/it]

In [ ]:
model.save_pretrained('fine-tuned-abstractive/')
tokenizer.save_pretrained('fine-tuned-abstractive/')

('Abstractive-fine-tunedmodel/tokenizer_config.json',
 'Abstractive-fine-tunedmodel/special_tokens_map.json',
 'Abstractive-fine-tunedmodel/spiece.model',
 'Abstractive-fine-tunedmodel/added_tokens.json',
 'Abstractive-fine-tunedmodel/tokenizer.json')

In [17]:
article=dataset['document'][2001]

In [18]:
model_path='fine-tuned-abstractive'
local_model=AutoModelForSeq2SeqLM.from_pretrained(model_path)
local_tokenizer=AutoTokenizer.from_pretrained(model_path)

In [19]:
test_ids=local_tokenizer('summarization: '+article,return_tensors='pt')

In [20]:
summary_ids=local_model.generate(test_ids['input_ids'],max_length=64)

In [21]:
summary=local_tokenizer.decode(summary_ids[0],skip_special_tokens=True)

In [22]:
summary

'Tranmere have been offered new terms, while Steve McNulty has been offered new terms.'

In [24]:
dataset['summary'][2001]

'Tranmere have released six players after their failure to win promotion back to the Football League.'

In [37]:
from datasets import load_metric

In [38]:
rouge=load_metric('rouge')

C:\Users\Sai Teja\AppData\Local\Temp\ipykernel_12060\1985591601.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge=load_metric('rouge')
C:\Users\Sai Teja\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\datasets\load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.2/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [25]:
def evaluate(data,idx):
    inputs=local_tokenizer('summarization: '+data['document'][idx],return_tensors='pt')
    summary=local_model.generate(inputs['input_ids'],max_length=64)
    summary=local_tokenizer.decode(summary[0],skip_special_tokens=True)
    return summary

In [26]:
predicted_summary=[evaluate(dataset,i) for i in range(10)]

Token indices sequence length is longer than the specified maximum sequence length for this model (543 > 512). Running this sequence through the model will result in indexing errors


In [33]:
predicted_summary

['The full cost of damage in Newton Stewart is still being assessed, a summarization has been made.',
 'A fire alarm went off at a hotel in Northern Ireland on Saturday and guests were asked to leave the hotel.',
 '- but the FIA has resigned the new elimination qualifying system for Bahrain.',
 'A former RAF officer has been charged with indecency with a child.',
 'A hospital in Istanbul has reported that a man who threatened to shoot himself and others hostage has been banned from carrying a gun.',
 'The Dragons fought back to a 2-0 win over Leinster in the Dublin Rugby Cup.',
 'A woman who died after a car collided with a tree in Croydon has been left in a critical condition, police say.',
 'A Belgian cyclist who died after a collision with a motorbike during a Belgian road racing race has said it will "co-operate" with all relevant authorities.',
 'Tottenham boss David Gundogan has said he will not rush his return to the Premier League after tearing knee ligaments in December.',
 'A

In [39]:
scores=rouge.compute(references=dataset['summary'][:10],predictions=predicted_summary)

In [97]:
for key in scores:
    print(f"{key}: {scores[key].high}")

rouge1: Score(precision=0.4983489304812834, recall=0.24900011727178903, fmeasure=0.45084176583853044)
rouge2: Score(precision=0.2162177677395068, recall=0.27349214191605495, fmeasure=0.22894427309320926)
rougeL: Score(precision=0.5596637506012506, recall=0.65207901004934253, fmeasure=0.61818940820475344)
rougeLsum: Score(precision=0.458069234006734, recall=0.40288747771836005, fmeasure=0.4185468969081501)
